### Imports

In [138]:
import os
import numpy as np
import pandas as pd

### Inputs

In [139]:
radnet_input_df = pd.read_csv("../inputs/radnet_input.csv")
radnet_input_efsr_edge = radnet_input_df['efsr_edge'].values

In [140]:
inference_output_df = pd.read_csv("../outputs/inference_output.csv")
inference_output_class = inference_output_df['class'].values
inference_output_confidence = inference_output_df['confidence'].values

In [141]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

### Fusion

In [142]:
w_model = 0.6
w_efsr = 0.4
w_sum = w_model + w_efsr

In [143]:
confidence_model = np.where(
    inference_output_class == 1,
    inference_output_confidence,
    1.0 - inference_output_confidence
)

In [144]:
efsr_min = 0.20
efsr_max = 0.98

confidence_efsr = (radnet_input_efsr_edge - efsr_min) / (efsr_max - efsr_min)
confidence_efsr = np.clip(confidence_efsr, 0.0, 1.0)

In [145]:
fused = (w_model * confidence_model + w_efsr * confidence_efsr) / w_sum

In [146]:
cls = (fused >= 0.5).astype(int)
conf = np.where(cls == 1, fused, 1.0 - fused)

### Apply

In [147]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'EFSR':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*62}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(edges_input):
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{radnet_input_efsr_edge[i]:<6.2f} "
          f"{cls[i]:<8} {CLASS_NAMES[int(cls[i])]:<8} {conf[i]:<10.2f}")

RSSI   ETX   Delay  Busy   EFSR   Classe   Status   Confiança 
--------------------------------------------------------------
-75    6.1   77     0.87   0.43   0        Ruim     0.88      
-54    2.7   19     0.41   0.91   1        Bom      0.94      
-81    4.8   127    0.89   0.79   0        Ruim     0.70      
-56    2.3   14     0.41   0.95   1        Bom      0.97      
-79    4.8   18     0.85   0.92   0        Ruim     0.63      
-61    1.5   6      0.27   0.91   1        Bom      0.96      
-66    4.5   79     0.87   0.39   0        Ruim     0.90      
-49    1.8   6      0.40   0.92   1        Bom      0.97      
-95    3.8   148    0.34   0.43   0        Ruim     0.88      
-55    1.6   11     0.29   0.88   1        Bom      0.94      
-49    1.1   7      0.55   0.91   1        Bom      0.96      
-72    6.3   99     0.79   0.93   0        Ruim     0.63      
-95    9.8   79     0.92   0.93   0        Ruim     0.63      
-82    3.9   34     0.69   0.93   0        Ruim     0.6

### Save

In [148]:
fusion_output = pd.DataFrame({
    'class': cls,
    'confidence': conf
})

os.makedirs("../outputs", exist_ok=True)
fusion_output.to_csv("../outputs/fusion_output.csv", index=False)

print("Fusão de Dados salva em ../outputs/fusion_output.csv")

Fusão de Dados salva em ../outputs/fusion_output.csv
